# Прогнозируем задержки самолетов

In [ ]:
#!pip install catboost lightgbm optuna -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.3/365.3 KB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.5/210.5 KB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 KB 8.0 MB/s eta 0:00:00


In [6]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import roc_auc_score
import pandas as pd
import numpy as np

from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier

In [2]:
RANDOM_STATE = 111
#DATASET_PATH = 'https://raw.githubusercontent.com/evgpat/edu_stepik_practical_ml/main/datasets/flight_delays_train.csv'

In [3]:
data = pd.read_csv('flight_delays_train.csv')

X = data.drop('dep_delayed_15min', axis=1)
y = data['dep_delayed_15min'] == 'Y'

X.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732
1,c-4,c-20,c-3,1548,US,PIT,MCO,834
2,c-9,c-2,c-5,1422,XE,RDU,CLE,416
3,c-11,c-25,c-6,1015,OO,DEN,MEM,872
4,c-10,c-7,c-6,1828,WN,MDW,OMA,423


Создайте список номеров колонок с категориальными признаками для бустингов

## Quiz
Какой длины получился список?

(подсказка: колонка `DepTime` числовая)

In [4]:
X.dtypes

Month            object
DayofMonth       object
DayOfWeek        object
DepTime           int64
UniqueCarrier    object
Origin           object
Dest             object
Distance          int64
dtype: object

In [23]:
# your code here
cat_features = np.where(X.dtypes != np.int64)[0]
cat_features.shape[0]

6

Разобъем данные на обучение и контроль

In [20]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.25, random_state=RANDOM_STATE)

In [21]:
Xtrain.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance
41207,c-4,c-18,c-1,1457,CO,EWR,TPA,998
28283,c-11,c-1,c-2,1225,UA,DEN,BOS,1754
34619,c-6,c-16,c-5,1650,YV,IAD,CAE,401
8789,c-5,c-18,c-4,923,AA,SLC,DFW,988
38315,c-2,c-14,c-2,1839,AA,STL,SAN,1558


## Модели с параметрами по умолчанию

Обучите CatBoost с гиперпараметрами по умолчанию.

## Quiz
Чему равен ROC-AUC на тестовых данных? Ответ округлите до сотых.

In [25]:
# your code here
model = CatBoostClassifier(random_seed=RANDOM_STATE, logging_level='Silent')
model.fit(Xtrain, ytrain, cat_features=cat_features)

In [32]:
pred = model.predict_proba(Xtest)
print(roc_auc_score(ytest, pred[:, 1]))

0.7669105405775934


Обучите LightGBM с гиперпараметрами по умолчанию.

## Quiz
Чему равен ROC-AUC на тестовых данных? Ответ округлите до сотых.

In [33]:
for c in X.columns:
    col_type = X[c].dtype
    if col_type == 'object' or col_type.name == 'category':
        Xtrain[c] = Xtrain[c].astype('category')
        Xtest[c] = Xtest[c].astype('category')

In [35]:
# your code here
model = LGBMClassifier(random_seed=RANDOM_STATE, verbose=-1)
model.fit(Xtrain, ytrain)

LGBMClassifier(random_seed=111, verbose=-1)

In [36]:
pred = model.predict_proba(Xtest)
print(roc_auc_score(ytest, pred[:, 1]))

0.7341149074685321


## Optuna

Выделим дополнительную валидационную выборку.

In [37]:
Xtrain_new, Xval, ytrain_new, yval = train_test_split(Xtrain, ytrain, test_size=0.25, random_state=RANDOM_STATE)

Создайте функцию objective_lgbm, в которой среди гиперпараметров

* num_leaves = trial.suggest_int("num_leaves", 10, 100)
* n_estimators = trial.suggest_int("n_estimators", 10, 1000)

подберите оптимальные, обучая LGBM на Xtrain_new, ytrain_new и проверяя качество (ROC-AUC) на Xval.

Используйте 30 эпох обучения Optuna.


In [39]:
import optuna

In [41]:
# your code here
def objective_lgbm(trial):
    num_leaves = trial.suggest_int("num_leaves", 10, 100)
    n_estimators = trial.suggest_int("n_estimators", 10, 1000)

    score = cross_val_score(LGBMClassifier(num_leaves=num_leaves, n_estimators=n_estimators),
                            Xtrain, ytrain, cv=3, scoring='roc_auc', n_jobs=-1).mean()
    return score


study = optuna.create_study(direction="maximize")
study.optimize(objective_lgbm, n_trials=30)

[I 2024-01-31 11:49:53,056] A new study created in memory with name: no-name-ab164459-19f0-42c1-89e1-eb0dc8813543
[I 2024-01-31 11:49:57,711] Trial 0 finished with value: 0.7150459027687016 and parameters: {'num_leaves': 30, 'n_estimators': 623}. Best is trial 0 with value: 0.7150459027687016.
[I 2024-01-31 11:50:03,788] Trial 1 finished with value: 0.7157546765075952 and parameters: {'num_leaves': 83, 'n_estimators': 513}. Best is trial 1 with value: 0.7157546765075952.
[I 2024-01-31 11:50:11,732] Trial 2 finished with value: 0.7152861565798818 and parameters: {'num_leaves': 81, 'n_estimators': 814}. Best is trial 1 with value: 0.7157546765075952.
[I 2024-01-31 11:50:18,956] Trial 3 finished with value: 0.7159246038892645 and parameters: {'num_leaves': 90, 'n_estimators': 537}. Best is trial 3 with value: 0.7159246038892645.
[I 2024-01-31 11:50:23,802] Trial 4 finished with value: 0.714086980285169 and parameters: {'num_leaves': 95, 'n_estimators': 306}. Best is trial 3 with value: 0.

In [42]:
study.best_params

{'num_leaves': 10, 'n_estimators': 145}

Обучите модель с найденными гиперпараметрами на Xtrain, ytrain и оцените ROC-AUC на тестовых данных.

In [44]:
# your code here
model = LGBMClassifier(**study.best_params)
model.fit(Xtrain, ytrain)

pred = model.predict_proba(Xtest)
print(roc_auc_score(ytest, pred[:, 1]))

[LightGBM] [Info] Number of positive: 14346, number of negative: 60654
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001670 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1095
[LightGBM] [Info] Number of data points in the train set: 75000, number of used features: 8
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.191280 -> initscore=-1.441714
[LightGBM] [Info] Start training from score -1.441714
0.7345959892385305


## Quiz

Чему равно количество деревьев в LGBM после подбора гиперпараметров?

## Работа над улучшением модели

* Попробуйте при помощи Optuna подобрать и другие гиперпарамтеры
* Также подберите гиперпараметры у CatBoost (а не только у LightGBM)

In [57]:
# your code here
def objective_lgb(trial):
    lgb_params = {
        'learning_rate': trial.suggest_float("learning_rate", 0.01, 1),
        'max_depth': trial.suggest_int("max_depth", 2, 16),
        'n_estimators': trial.suggest_int("n_estimators", 10, 1000),
        'subsample': trial.suggest_float("subsample", 0.1, 1),
        'num_leaves': trial.suggest_int("num_leaves", 10, 500),
        'random_state': RANDOM_STATE, 
        'verbose': -1
    }
    
    lgb=LGBMClassifier(**lgb_params)
    lgb.fit(Xtrain_new, ytrain_new,
        eval_set=[(Xval, yval)], eval_metric='auc')
    
    y_pred_val = lgb.predict_proba(Xval)[:,1]
    auc_val = roc_auc_score(yval, y_pred_val)
    return auc_val


study = optuna.create_study(direction="maximize")
study.optimize(objective_lgb, n_trials=30)

[I 2024-01-31 12:33:04,871] A new study created in memory with name: no-name-abcae979-457c-4f8c-9381-ab1771b7b327
[I 2024-01-31 12:33:06,951] Trial 0 finished with value: 0.7272018200827767 and parameters: {'learning_rate': 0.06963059801644719, 'max_depth': 16, 'n_estimators': 144, 'subsample': 0.3968627359163287, 'num_leaves': 290}. Best is trial 0 with value: 0.7272018200827767.
[I 2024-01-31 12:33:10,028] Trial 1 finished with value: 0.710498053670397 and parameters: {'learning_rate': 0.3882345085021755, 'max_depth': 12, 'n_estimators': 338, 'subsample': 0.8892604117000764, 'num_leaves': 137}. Best is trial 0 with value: 0.7272018200827767.
[I 2024-01-31 12:33:13,018] Trial 2 finished with value: 0.7189842458988785 and parameters: {'learning_rate': 0.06635075616466661, 'max_depth': 6, 'n_estimators': 509, 'subsample': 0.6626529456820859, 'num_leaves': 86}. Best is trial 0 with value: 0.7272018200827767.
[I 2024-01-31 12:33:14,952] Trial 3 finished with value: 0.6970762150160631 and 

In [58]:
study.best_params

{'learning_rate': 0.2089613462143135,
 'max_depth': 14,
 'n_estimators': 757,
 'subsample': 0.2584543328511252,
 'num_leaves': 482}

In [59]:
model = LGBMClassifier(**study.best_params)
model.fit(Xtrain, ytrain)

pred = model.predict_proba(Xtest)
print(roc_auc_score(ytest, pred[:, 1]))

[LightGBM] [Info] Number of positive: 14346, number of negative: 60654
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001642 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1095
[LightGBM] [Info] Number of data points in the train set: 75000, number of used features: 8
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.191280 -> initscore=-1.441714
[LightGBM] [Info] Start training from score -1.441714
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
0.7506452587218652


## Quiz

Поделитесь своими результатами!